<a class="anchor" id="0.1"></a>

## Table of Contents
1. [Import labriraries and variables](#1)
2. [О датасете/EDA](#2)
3. [Feature Engineering](#3)
4. [Train models](#4)

## Import labriraries and variables
<a class="anchor" id="1"></a>

[Back to Table of Contents](#0.1)

In [2]:
!pip install kaggle_metrics

In [3]:
import pandas as pd
import numpy as np
import pyarrow as pa
import matplotlib as plt

from kaggle_metrics.utils import check_shapes, align_shape
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier


In [5]:
def weighted_mean_absolute_error(y_true, y_pred, weights):

    '''

    Weighted mean absolute error.

    Parameters
    ----------
    y_true: ndarray
        Ground truth
    y_pred: ndarray
        Array of predictions

    Returns
    -------
    rmsle: float
        Weighted mean absolute error

    References
    ----------
    .. [1] https://www.kaggle.com/wiki/WeightedMeanAbsoluteError

    '''

    # Check shapes
    y_true, y_pred = align_shape(y_true, y_pred)
    check_shapes(y_true, y_pred)

    return (weights * np.abs(y_true - y_pred)).mean()

In [6]:
# variables 
main_dir = "/kaggle/input/alfa-challenge/"

In [7]:
transaction = pd.read_parquet(main_dir + "df_transaction.pa", engine="pyarrow")
train = pd.read_parquet(main_dir + "train.pa", engine="pyarrow")

## О датасете/EDA
<a class="anchor" id="2"></a>

[Back to Table of Contents](#0.1)

### DataSet "Transaction"
client_num - номер клиента

date_time - время транзакции

mcc_code - МСС код

merchant_name - hash имени мерчанта

amount - сумма транзакции

In [8]:
transaction

,client_num,date_time,mcc_code,merchant_name,amount
0,0,2024-07-18 16:04:00,8099,a011100358d0f73ea8f3e860ef5564e3ba9cb217b7b90c...,2900
1,0,2024-07-22 16:31:00,5411,f3855606fc7244ec2f37ea01a4b2b66933d0e965bf4aec...,455
2,0,2024-07-24 16:23:00,5541,786270fa33ad4ac2a3c0e52e888005aa7f98beadbf8986...,1003
3,0,2024-07-28 15:51:00,5691,54887ad4a8df7e260a3ac85e59128a947c50d4423f6330...,1480
4,0,2024-07-28 18:00:00,5331,21617559a372c7cca155208c87be6c84ce97b5f8775589...,88
...,...,...,...,...,...
13508150,109142,2024-08-19 21:32:00,6011,01784811094a8bd592cb35ee21e98c934839341e2b9d14...,14000
13508151,109142,2024-08-19 21:40:00,6011,01784811094a8bd592cb35ee21e98c934839341e2b9d14...,24000
13508152,109142,2024-08-19 21:46:00,6011,01784811094a8bd592cb35ee21e98c934839341e2b9d14...,23000
13508153,109142,2024-08-19 22:04:00,6011,01784811094a8bd592cb35ee21e98c934839341e2b9d14...,32000


In [24]:
transaction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13508155 entries, 0 to 13508154
Data columns (total 5 columns):
 #   Column         Dtype         
---  ------         -----         
 0   client_num     int64         
 1   date_time      datetime64[us]
 2   mcc_code       object        
 3   merchant_name  object        
 4   amount         int64         
dtypes: datetime64[us](1), int64(2), object(2)
memory usage: 515.3+ MB


In [10]:
transaction.isnull().sum()

client_num       0
date_time        0
mcc_code         0
merchant_name    0
amount           0
dtype: int64

In [9]:
transaction_copy = transaction.copy()

In [11]:
transaction_copy['month'] = transaction['date_time'].dt.to_period('M')

In [70]:
transaction_copy.groupby('month')['client_num'].count()

month
2024-07    4441222
2024-08    4577178
2024-09    4489751
2024-10          4
Freq: M, Name: client_num, dtype: int64

В основном покупки совершили в июле месяце. Можно будет проверить, есть ли клиенты из июля в августе и в сентябре.

In [71]:
transaction_copy.groupby(['month', 'client_num'])['amount'].sum()

month    client_num
2024-07  0                7261
         1              422749
         2              114647
         3             1483913
         4               91422
                        ...   
2024-09  109141            378
2024-10  13168            3280
         26937             132
         78137              64
         93720              31
Name: amount, Length: 298590, dtype: int64

In [73]:
transaction_copy

,client_num,date_time,mcc_code,merchant_name,amount,month,sum_amount
0,0,2024-07-18 16:04:00,8099,a011100358d0f73ea8f3e860ef5564e3ba9cb217b7b90c...,2900,2024-07,7261
1,0,2024-07-22 16:31:00,5411,f3855606fc7244ec2f37ea01a4b2b66933d0e965bf4aec...,455,2024-07,7261
2,0,2024-07-24 16:23:00,5541,786270fa33ad4ac2a3c0e52e888005aa7f98beadbf8986...,1003,2024-07,7261
3,0,2024-07-28 15:51:00,5691,54887ad4a8df7e260a3ac85e59128a947c50d4423f6330...,1480,2024-07,7261
4,0,2024-07-28 18:00:00,5331,21617559a372c7cca155208c87be6c84ce97b5f8775589...,88,2024-07,7261
...,...,...,...,...,...,...,...
13508150,109142,2024-08-19 21:32:00,6011,01784811094a8bd592cb35ee21e98c934839341e2b9d14...,14000,2024-08,341100
13508151,109142,2024-08-19 21:40:00,6011,01784811094a8bd592cb35ee21e98c934839341e2b9d14...,24000,2024-08,341100
13508152,109142,2024-08-19 21:46:00,6011,01784811094a8bd592cb35ee21e98c934839341e2b9d14...,23000,2024-08,341100
13508153,109142,2024-08-19 22:04:00,6011,01784811094a8bd592cb35ee21e98c934839341e2b9d14...,32000,2024-08,341100


In [76]:
transaction_copy

,client_num,month,sum_amount
0,0,2024-07,7261
132,1,2024-07,422749
372,2,2024-07,114647
672,3,2024-07,1483913
819,4,2024-07,91422
...,...,...,...
13508075,109138,2024-08,131817
13508091,109139,2024-07,2996
13508106,109140,2024-07,369
13508124,109141,2024-07,12050


In [52]:
filtered_data = transaction_copy.groupby(['month', 'client_num']).agg({
    'amount': ['sum', 'mean', 'min', 'max'],
    'mcc_code': 'count'
}).reset_index()
filtered_data

KeyError: "Column(s) ['amount', 'mcc_code'] do not exist"

In [8]:
transaction_copy.mcc_code.unique().shape[0], transaction_copy.merchant_name.unique().shape[0]

(320, 666279)

In [11]:
transaction_copy.groupby(['client_num'])['mcc_code'].count()

client_num
0         132
1         240
2         300
3         147
4         122
         ... 
109138     16
109139     15
109140     18
109141     16
109142     15
Name: mcc_code, Length: 109143, dtype: int64

Тут, скорее всего можно понять, как много человек тратит деньги в различных категориях

In [13]:
transaction_copy.groupby(['mcc_code'])['merchant_name'].count()

mcc_code
0742     6688
0763      239
0780     1209
1520      123
1711      214
        ...  
9311     7480
9390    17579
9399    10437
9402    15032
9406     7465
Name: merchant_name, Length: 320, dtype: int64

А тут можно можно понять, где больше всего продавцов в тех или иных категориях

In [ ]:
transaction_copy.groupby('month')['client_num'].count()

### Dataset "Train"
client_num - номер клиента

target - группа риска

In [28]:
train

,client_num,target
0,94779,3
1,17279,0
2,5717,2
3,27471,1
4,72725,0
...,...,...
69995,107219,1
69996,108682,1
69997,93497,3
69998,14344,6


In [42]:
train.client_num.unique().shape[0]

70000

In [43]:
transaction_copy.client_num.unique().shape[0]

109143

In [78]:
train_merge = train.merge(transaction_copy, on='client_num', how='left')
# train_merge.drop(['date_time'], axis=1, inplace=True)

In [79]:
train_merge.sort_values(by='client_num')

,client_num,target,month,sum_amount
54429,1,4,2024-07,422749
69304,2,5,2024-07,114647
54543,3,3,2024-07,1483913
39100,4,5,2024-07,91422
19645,5,2,2024-07,29600
...,...,...,...,...
27657,109136,3,2024-07,9238
53872,109138,2,2024-08,131817
29655,109139,0,2024-07,2996
67622,109141,0,2024-07,12050


In [83]:
# train_merge.to_csv('new_train', index=False)
train_merge.to_parquet('new_tr.parquet')

## Feature Engineering
<a class="anchor" id="3"></a>

[Back to Table of Contents](#0.1)

In [13]:
transaction_copy['sum_amount'] = transaction_copy.groupby(['month', 'client_num'])['amount'].transform('sum')

In [14]:
transaction_copy.groupby('client_num')['mcc_code'].count()

client_num
0         132
1         240
2         300
3         147
4         122
         ... 
109138     16
109139     15
109140     18
109141     16
109142     15
Name: mcc_code, Length: 109143, dtype: int64

In [15]:
transaction_copy['sum_buys'] = transaction_copy.groupby('client_num')['mcc_code'].count()

In [16]:
transaction_copy

,client_num,date_time,mcc_code,merchant_name,amount,month,sum_amount,sum_buys
0,0,2024-07-18 16:04:00,8099,a011100358d0f73ea8f3e860ef5564e3ba9cb217b7b90c...,2900,2024-07,7261,132.0
1,0,2024-07-22 16:31:00,5411,f3855606fc7244ec2f37ea01a4b2b66933d0e965bf4aec...,455,2024-07,7261,240.0
2,0,2024-07-24 16:23:00,5541,786270fa33ad4ac2a3c0e52e888005aa7f98beadbf8986...,1003,2024-07,7261,300.0
3,0,2024-07-28 15:51:00,5691,54887ad4a8df7e260a3ac85e59128a947c50d4423f6330...,1480,2024-07,7261,147.0
4,0,2024-07-28 18:00:00,5331,21617559a372c7cca155208c87be6c84ce97b5f8775589...,88,2024-07,7261,122.0
...,...,...,...,...,...,...,...,...
13508150,109142,2024-08-19 21:32:00,6011,01784811094a8bd592cb35ee21e98c934839341e2b9d14...,14000,2024-08,341100,NaN
13508151,109142,2024-08-19 21:40:00,6011,01784811094a8bd592cb35ee21e98c934839341e2b9d14...,24000,2024-08,341100,NaN
13508152,109142,2024-08-19 21:46:00,6011,01784811094a8bd592cb35ee21e98c934839341e2b9d14...,23000,2024-08,341100,NaN
13508153,109142,2024-08-19 22:04:00,6011,01784811094a8bd592cb35ee21e98c934839341e2b9d14...,32000,2024-08,341100,NaN


In [ ]:
transaction_copy.drop(columns=['date_time', 'mcc_code', 'merchant_name', 'amount'], inplace=True)
transaction_copy.drop_duplicates(subset=['client_num'], inplace=True)

In [95]:
train_merge['month'] = train_merge['month'].astype(str)

In [97]:
train_merge['month']

0        2024-07
1        2024-09
2        2024-07
3        2024-07
4        2024-07
          ...   
69995    2024-07
69996    2024-07
69997    2024-07
69998    2024-07
69999    2024-07
Name: month, Length: 70000, dtype: object

In [98]:
mapping = {
    '2024-07': 1,
    '2024-08': 2,
    '2024-09': 3
}
train_merge_mapping = train_merge.copy()
train_merge_mapping['month'] = train_merge_mapping['month'].map(mapping)

In [99]:
train_merge_mapping

,client_num,target,month,sum_amount
0,94779,3,1,9700
1,17279,0,3,7787
2,5717,2,1,3581
3,27471,1,1,68904
4,72725,0,1,43733
...,...,...,...,...
69995,107219,1,1,10
69996,108682,1,1,156
69997,93497,3,1,21500
69998,14344,6,1,2291


In [102]:
X = train_merge_mapping.drop(columns=['target'])
y = train_merge_mapping['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Train models
<a class="anchor" id="4"></a>

[Back to Table of Contents](#0.1)

### BaseLine


In [103]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(random_state=42, n_estimators=100)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [116]:
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:, 1]
weights = np.ones(len(y_test))

absolute_errors = np.abs(y_test - y_pred_proba)
weighted_absolute_errors = weights * absolute_errors

# Итоговая метрика WMAE
wmae = np.sum(weighted_absolute_errors) / np.sum(weights)
print("WMAE:", wmae)

WMAE: 2.0785657142857143


### Tuning parameters

In [117]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

grid_search = GridSearchCV(
    RandomForestClassifier(random_state=42),
    param_grid,
    cv=3,
    scoring='accuracy'
)
grid_search.fit(X_train, y_train)
best_model = grid_search.best_estimator_
print("Лучшие параметры:", grid_search.best_params_)

KeyboardInterrupt: 